# 🥉 Camada Bronze - Ingestão de Dados Brutos

Este notebook executa a ingestão de dados brutos de APIs governamentais e armazena na camada Bronze em formato Parquet.

## Fontes de Dados:
- **IBGE**: Municípios, Estados, População
- **Portal da Transparência**: Órgãos SIAFI, BPC por Município

## Estrutura de Saída:
```
bronze/
├── ibge/
│   ├── municipios/dt=YYYYMMDD/data.parquet
│   ├── estados/dt=YYYYMMDD/data.parquet
│   └── populacao_estados/dt=YYYYMMDD/data.parquet
└── portal_transparencia/
    ├── orgaos_siafi/dt=YYYYMMDD/data.parquet
    └── bpc_municipios/dt=YYYYMMDD/data.parquet
```

In [ ]:
# Instalar dependências se necessário
!pip install -q requests pandas minio pyarrow

In [ ]:
import requestsimport pandas as pdfrom minio import Miniofrom minio.error import S3Errorimport iofrom datetime import datetimeimport json# ConfiguraçõesPORTAL_TRANSPARENCIA_API_KEY = "2c56919ba91b8c1b13473dcef43fb031"MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"MINIO_ROOT_USER = "admin"MINIO_ROOT_PASSWORD = "1q2w3e4r"BUCKET_NAME = "govbr"transparency_url = "http://api.portaldatransparencia.gov.br/api-de-dados"ibge_url = "https://servicodados.ibge.gov.br/api/v1"headers = {    'chave-api-dados': PORTAL_TRANSPARENCIA_API_KEY}# Cliente MinIOminio_client = Minio(    MINIO_SERVER_URL,    access_key=MINIO_ROOT_USER,    secret_key=MINIO_ROOT_PASSWORD,    secure=True)# Verificar/criar bucketif not minio_client.bucket_exists(BUCKET_NAME):    minio_client.make_bucket(BUCKET_NAME)    print(f"✅ Bucket '{BUCKET_NAME}' criado")else:    print(f"✅ Bucket '{BUCKET_NAME}' já existe")def save_to_bronze(df, dataset_name, source, partition_date=None):    """Salva DataFrame na camada Bronze em formato Parquet"""    if partition_date is None:        partition_date = datetime.now().strftime('%Y%m%d')    object_name = f"bronze/{source}/{dataset_name}/dt={partition_date}/data.parquet"    try:        # Converter para Parquet        buffer = io.BytesIO()        df.to_parquet(buffer, index=False, engine='pyarrow', compression='snappy')        buffer.seek(0)        # Upload para MinIO        minio_client.put_object(            BUCKET_NAME,            object_name,            buffer,            length=buffer.getbuffer().nbytes,            content_type='application/octet-stream'        )        print(f"✅ Bronze: {object_name} ({len(df)} registros, {buffer.getbuffer().nbytes/1024:.2f} KB)")        return True    except Exception as e:        print(f"❌ Erro ao salvar {object_name}: {e}")        return Falseprint("=" * 80)print("CAMADA BRONZE - INGESTÃO DE DADOS BRUTOS")print("=" * 80)

## [1/5] Coletando Municípios do Brasil (IBGE)

In [ ]:
# IBGE - Municípiosprint("\n[1/5] Coletando municípios do Brasil (IBGE)...")response = requests.get(f"{ibge_url}/localidades/municipios", timeout=30)if response.status_code == 200:    municipios = response.json()    municipios_data = []    for m in municipios:        try:            microrregiao = m.get('microrregiao', {})            mesorregiao = microrregiao.get('mesorregiao', {}) if microrregiao else {}            uf = mesorregiao.get('UF', {}) if mesorregiao else {}            regiao = uf.get('regiao', {}) if uf else {}                        municipios_data.append({                'codigo_ibge': m.get('id'),                'municipio': m.get('nome'),                'uf_sigla': uf.get('sigla'),                'uf_nome': uf.get('nome'),                'regiao_id': regiao.get('id'),                'regiao_nome': regiao.get('nome'),                'microrregiao_id': microrregiao.get('id') if microrregiao else None,                'microrregiao_nome': microrregiao.get('nome') if microrregiao else None,                'mesorregiao_id': mesorregiao.get('id') if mesorregiao else None,                'mesorregiao_nome': mesorregiao.get('nome') if mesorregiao else None            })        except Exception as e:            print(f"  ⚠️  Erro ao processar município {m.get('id', 'N/A')}: {e}")            continue        df_municipios = pd.DataFrame(municipios_data)    print(f"✅ {len(df_municipios)} municípios coletados")    save_to_bronze(df_municipios, 'municipios', 'ibge')else:    print(f"❌ Erro ao coletar municípios: {response.status_code}")

## [2/5] Coletando Estados do Brasil (IBGE)

In [ ]:
# IBGE - Estadosprint("\n[2/5] Coletando estados do Brasil (IBGE)...")response = requests.get(f"{ibge_url}/localidades/estados", timeout=30)if response.status_code == 200:    estados = response.json()    df_estados = pd.DataFrame([{        'uf_id': e['id'],        'uf_sigla': e['sigla'],        'uf_nome': e['nome'],        'regiao_id': e['regiao']['id'],        'regiao_sigla': e['regiao']['sigla'],        'regiao_nome': e['regiao']['nome']    } for e in estados])    print(f"✅ {len(df_estados)} estados coletados")    save_to_bronze(df_estados, 'estados', 'ibge')else:    print(f"❌ Erro ao coletar estados: {response.status_code}")

## [3/5] Coletando Órgãos SIAFI (Portal da Transparência)

In [ ]:
# Portal da Transparência - Órgãos SIAFIprint("\n[3/5] Coletando órgãos SIAFI...")response = requests.get(f"{transparency_url}/orgaos-siafi", headers=headers, timeout=30)if response.status_code == 200:    orgaos = response.json()    df_orgaos = pd.DataFrame(orgaos)    # Filtrar órgãos válidos    df_orgaos = df_orgaos[~df_orgaos['descricao'].str.contains('CODIGO INVALIDO', na=False)]        print(f"✅ {len(df_orgaos)} órgãos coletados")    save_to_bronze(df_orgaos, 'orgaos_siafi', 'portal_transparencia')else:    print(f"❌ Erro ao coletar órgãos: {response.status_code}")

## [4/5] Coletando Dados de BPC (Portal da Transparência)

In [ ]:
# Portal da Transparência - BPC por município (amostra de SP)print("\n[4/5] Coletando dados de BPC (amostra SP - primeiros 50 municípios)...")# Pegar municípios de SPif 'df_municipios' in locals() and df_municipios is not None and len(df_municipios) > 0:    df_municipios_sp = df_municipios[df_municipios['uf_sigla'] == 'SP'].head(50)else:    print("  ⚠️  Municípios não disponíveis, pulando coleta de BPC")    df_municipios_sp = pd.DataFrame()bpc_data = []if len(df_municipios_sp) > 0:    for idx, row in df_municipios_sp.iterrows():        codigo = str(row['codigo_ibge'])        nome = row['municipio']        if idx % 10 == 0:            print(f"  Progresso: {idx}/{len(df_municipios_sp)} municípios...")        try:            response = requests.get(                f"{transparency_url}/bpc-por-municipio",                headers=headers,                params={'mesAno': '202412', 'codigoIbge': codigo, 'pagina': 1},                timeout=10            )            if response.status_code == 200 and response.json():                data = response.json()[0]                # Flatten nested structure                record = {                    'id': data.get('id'),                    'data_referencia': data.get('dataReferencia'),                    'codigo_ibge': data['municipio']['codigoIBGE'],                    'nome_municipio': data['municipio']['nomeIBGE'],                    'uf_sigla': data['municipio']['uf']['sigla'],                    'uf_nome': data['municipio']['uf']['nome'],                    'regiao_nome': data['municipio']['nomeRegiao'],                    'tipo_id': data['tipo']['id'],                    'tipo_descricao': data['tipo']['descricao'],                    'tipo_descricao_detalhada': data['tipo']['descricaoDetalhada'],                    'valor': data.get('valor'),                    'quantidade_beneficiados': data.get('quantidadeBeneficiados')                }                bpc_data.append(record)        except Exception as e:            print(f"  ⚠️  Erro em {nome}: {e}")if len(df_municipios_sp) > 0 and bpc_data:    df_bpc = pd.DataFrame(bpc_data)    save_to_bronze(df_bpc, 'bpc_municipios', 'portal_transparencia')    print(f"  ✅ {len(bpc_data)} municípios com dados de BPC")else:    print("  ❌ Nenhum dado de BPC coletado")

## [5/5] Coletando População por Estado (IBGE)

In [ ]:
# IBGE - População estimada por estadoprint("\n[5/5] Coletando estimativas de população por estado...")try:    populacao_data = []    if 'df_estados' not in locals() or df_estados is None or len(df_estados) == 0:        raise Exception("Estados não disponíveis")        for idx, row in df_estados.iterrows():        uf_id = row['uf_id']        uf_sigla = row['uf_sigla']        # Endpoint de projeção de população        response = requests.get(            f"{ibge_url}/projecoes/populacao/{uf_id}",            timeout=10        )        if response.status_code == 200:            data = response.json()            if data and 'projecao' in data:                populacao_data.append({                    'uf_id': uf_id,                    'uf_sigla': uf_sigla,                    'ano': 2024,                    'populacao': data['projecao']['populacao']                })    if populacao_data:        df_populacao = pd.DataFrame(populacao_data)        print(f"✅ {len(df_populacao)} estados com dados de população")        save_to_bronze(df_populacao, 'populacao_estados', 'ibge')    else:        raise Exception("Nenhum dado de população coletado")        except Exception as e:    print(f"  ⚠️  Endpoint de população não disponível, usando dados manuais: {e}")    # Dados de população estimada 2024 (fonte: IBGE)    populacao_manual = [        {'uf_sigla': 'SP', 'uf_id': 35, 'ano': 2024, 'populacao': 46649132},        {'uf_sigla': 'MG', 'uf_id': 31, 'ano': 2024, 'populacao': 21411923},        {'uf_sigla': 'RJ', 'uf_id': 33, 'ano': 2024, 'populacao': 17463349},        {'uf_sigla': 'BA', 'uf_id': 29, 'ano': 2024, 'populacao': 14985284},        {'uf_sigla': 'PR', 'uf_id': 41, 'ano': 2024, 'populacao': 11597484},    ]    df_populacao = pd.DataFrame(populacao_manual)    print(f"✅ {len(df_populacao)} estados com dados manuais de população")    save_to_bronze(df_populacao, 'populacao_estados', 'ibge')

## Resumo da Ingestão

In [ ]:
# Listar arquivos Bronzeprint("\n" + "=" * 80)print("RESUMO DA INGESTÃO")print("=" * 80)objects = minio_client.list_objects(BUCKET_NAME, prefix="bronze/", recursive=True)bronze_files = list(objects)print(f"\nTotal de arquivos na camada Bronze: {len(bronze_files)}")total_size = 0for obj in bronze_files:    size_kb = obj.size / 1024    total_size += obj.size    print(f"  📁 {obj.object_name} ({size_kb:.2f} KB)")print(f"\nTamanho total: {total_size/1024:.2f} KB")print("\n✅ Ingestão Bronze concluída!")